In [9]:
import cv2 as cv
import numpy as np
import random
from matplotlib import pyplot as plt
import time
import imutils
from collections import deque
import cvzone
import math
from cvzone.HandTrackingModule import HandDetector

In [10]:
cap = cv.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
 
detector = HandDetector(detectionCon=0.8, maxHands=1)

In [11]:
class SnakeGame:
    def __init__(self, foodpath):
        self.points = []
        self.lengths= []
        self.cur_length = 0
        self.max_length = 150
        self.ini_score = 0
        self.gameOver = False
        self.Head = 0,0
        self.img_food = cv.imread(foodpath,cv.IMREAD_UNCHANGED)
        self.height , self.width, _ = self.img_food.shape
        self.foodPoint = 0, 0
        self.randomFoodLocation()
 

    #locate the food randomly
    def randomFoodLocation(self):
        self.foodPoint = random.randint(100, 1000), random.randint(100, 600)

    #the snake is updated 
    def update(self, imgMain, currentHead):
 
        if self.gameOver:
            cvzone.putTextRect(imgMain, "Game Over", [300, 400],
                               scale=7, thickness=5, offset=20)
            cvzone.putTextRect(imgMain, f'Your Score: {self.ini_score}', [300, 550],
                               scale=7, thickness=5, offset=20)
        else:
            px, py = self.Head
            cx, cy = currentHead
 
            self.points.append([cx, cy]) 
            distance = math.hypot(cx - px, cy - py) 
            self.lengths.append(distance)
            self.cur_length += distance
            self.Head = cx, cy #updating the head with the coordinates of the new head
 
            # length reduction 
            if self.cur_length > self.max_length:
                for i, length in enumerate(self.lengths):
                    self.cur_length -= length
                    self.lengths.pop(i)
                    self.points.pop(i)
                    if self.cur_length < self.max_length:
                        break
 
            # Check if snake ate the Food
            rx, ry = self.foodPoint
            if rx - self.width // 2 < cx < rx + self.width // 2 and \
                    ry - self.height // 2 < cy < ry + self.height // 2:
                self.randomFoodLocation()
                self.max_length += 50
                self.ini_score += 1
                print(self.ini_score)
 
            # Draw Snake
            if self.points:
                for i, point in enumerate(self.points):
                    if i != 0:
                        cv.line(imgMain, tuple(self.points[i - 1]), tuple(self.points[i]), (0, 0, 255), 20)
                cv.circle(imgMain, tuple(self.points[-1]), 20, (0, 255, 0), -1) #head
 
            # Draw Food
            imgMain = cvzone.overlayPNG(imgMain, self.img_food,
                                        (rx - self.width // 2, ry - self.height // 2))
 
            cvzone.putTextRect(imgMain, f'Score: {self.ini_score}', [50, 80],
                               scale=3, thickness=3, offset=10)
 
            # Check for Collision
            pts = np.array(self.points[:-2], np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv.polylines(imgMain, [pts], False, (0, 255, 0), 3)
            minDist = cv.pointPolygonTest(pts, (cx, cy), True)
 
            if -1 <= minDist <= 1:
                print("Hit")
                self.gameOver = True
                self.points = []  # all points of the snake
                self.lengths = []  # distance between each point
                self.cur_length = 0  # total length of the snake
                self.max_length = 150  # total allowed Length
                self.Head = 0, 0  # previous head point
                self.randomFoodLocation()
 
        return imgMain


In [12]:
img_path = "E:\iti-computer vision G1\project2\donut (3).png"
game = SnakeGame(img_path)
while True:
    ret,frame = cap.read()
    frame = cv.flip(frame, 1)
    hands, frame = detector.findHands(frame, flipType=False)
 
    if hands:
        lmList = hands[0]['lmList']
        pointIndex = lmList[8][0:2]
        frame = game.update(frame, pointIndex)
    cv.imshow("Image", frame)
    key = cv.waitKey(1)
    if key == ord('q'):
        game.gameOver = False
        break

cv.destroyAllWindows()

Hit
